# Dataset analyzer 

The aim is to inspect the data distribution

In [2]:
import torch
import pandas as pd
import transformers
import numpy as np
from transformers import TextClassificationPipeline
from IPython.display import display, HTML
from datasets import load_from_disk
from sklearn.preprocessing import LabelEncoder

words = [
    "white",
    "like",
    "nigger",
    "people",
    "women",
    "kike",
    "jews",
    "ghetto",
    "fucking",
    "retarded",
    "one",
    "black",
    "muslim",
    "fuck",
    "hate"
]


dataset = load_from_disk('/home/blanca/Documents/Thesis_Code/code/saved_datasets/dataset_light')
dataset = dataset["train"]
data = pd.DataFrame({"text": dataset["post_tokens"], "label": dataset["final_label"]})

def search_string(s, search):
    return search in str(s).lower()

def search_perfect_match(s, search):
    return str(s).lower() == search.lower()

encoder = LabelEncoder()
encoder.classes_ = np.load('/home/blanca/Documents/Thesis_Code/Dataset/classes.npy',allow_pickle=True)
id2label = {idx: label for idx, label in enumerate(encoder.classes_)}
label2id = {label: idx for idx, label in enumerate(encoder.classes_)}


true_label_list = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = transformers.AutoTokenizer.from_pretrained('blancasavi/pretrained_light')
model = transformers.AutoModelForSequenceClassification.from_pretrained("blancasavi/pretrained_light",num_labels=len(encoder.classes_), id2label=id2label, label2id=label2id).cuda()

for i in range(len(words)):
    mask = data.apply(lambda x: x.map(lambda s: search_string(s, words[i])))
    filtered_df = data.loc[mask.any(axis=1)]
    #print(filtered_df)
    
    
    filtered_hs = filtered_df[filtered_df['label']==0]
    filtered_nor = filtered_df[filtered_df['label']==1]
    filtered_off = filtered_df[filtered_df['label']==2]
    
    hs = len(filtered_hs.index.tolist())
    nor = len(filtered_nor.index.tolist())
    off = len(filtered_off.index.tolist())

    result_tuple = (words[i], hs, nor, off)
    true_label_list.append(result_tuple)


df_truelabel = pd.DataFrame(true_label_list, columns=['Word (T)', 'Hatespeech', 'Normal', 'Offensive'])


html_true = df_truelabel.to_html(index=False)



combined_html = f"""
<div style="display: flex; justify-content: space-between;">
    <div style="flex: 1; padding-right: 10px;">
        <h3>True Labels</h3>
        {html_true}
    </div>

</div>
"""

# Display the combined HTML
display(HTML(combined_html))



/home/blanca/miniconda3/envs/thesis2/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Word (T),Hatespeech,Normal,Offensive
white,750,1387,588
like,530,580,487
nigger,1618,152,188
people,281,558,297
women,203,514,274
kike,760,17,58
jews,355,145,219
ghetto,116,263,360
fucking,283,237,195
retarded,124,97,486
